# Texas Tow Trucks

We're going to scrape some [tow trucks in Texas](https://www.tdlr.texas.gov/tools_search/).

## Import your imports

In [89]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

driver=webdriver.Chrome()
driver.get('https://www.tdlr.texas.gov/tools_search/')

## Search for the TLDR Number `006179570C`, and scrape the information on that company

Using [license information system](https://www.tdlr.texas.gov/tools_search/), find information about the tow truck number above, displaying the

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

If you can't figure a 'nice' way to locate something, your two last options might be:

- **Find a "parent" element, then dig inside**
- **Find all of a type of element** (like we did with `td` before) and get the `[0]`, `[1]`, `[2]`, etc
- **XPath** (inspect an element, Copy > Copy XPath)

These kinds of techniques tend to break when you're on other result pages, but... maybe not! You won't know until you try.

> - *TIP: When you use xpath, you CANNOT use double quotes or Python will get confused. Use single quotes.*
> - *TIP: You can clean your data up if you want to, or leave it dirty to clean later*
> - *TIP: The address part can be tough, but you have a few options. You can use a combination of `.split` and list slicing to clean it now, or clean it later in the dataframe with regular expressions. Or other options, too, probably*

In [90]:
# find by tldr
tldr_button = driver.find_element_by_id('mcrbutton')
tldr_button.click()

# enter tldr number
tldr_input = driver.find_element_by_id('mcrdata')
tldr_input.send_keys('006179570C')

# click search
search_button = driver.find_element_by_id('submit3')
search_button.click()

In [91]:
# find business name
business_name = driver.find_elements_by_tag_name('td')[5].text
business_name.split(":")

# find owner/ operator
owner = driver.find_elements_by_tag_name('td')[7].text
owner

# find phone number
phone = driver.find_elements_by_tag_name('td')[9].text
phone

# find license status
status = driver.find_elements_by_tag_name('td')[12].text
status

# find physical address
address_info = driver.find_elements_by_tag_name('td')[14].text
address = address_info.split("\n")[5:7]
address

['13619 BRETT JACKSON RD', 'FORT WORTH, TX. 76179']

# Adapt this to work inside of a single cell

Double-check that it works. You want it to print out all of the details.

In [92]:
# find business name
try:
    business_name = driver.find_elements_by_tag_name('td')[5].text
    business = business_name.split(":")[1]
    print(business)
except:
    print("Address is wonky.")

# find owner/ operator
try:
    owner = driver.find_elements_by_tag_name('td')[7].text
    print(owner)
except:
    print("Owner is wonky.")

# find phone number
try:
    phone = driver.find_elements_by_tag_name('td')[9].text
    print(phone)
except:
    print("Phone number is wonky.")

# find license status
try:
    status = driver.find_elements_by_tag_name('td')[12].text
    print(status)
except:
    print("License status is wonky.")

# find physical address
try:
    address_info = driver.find_elements_by_tag_name('td')[14].text
    address = address_info.split("\n")[5:7]
    print(address)
except:
    print("Address is wonky.")

   B.D. SMITH TOWING
Owner/Officer:   BRANDT SMITH / OWNER
Phone:   8173330706
Status:  Expired
['13619 BRETT JACKSON RD', 'FORT WORTH, TX. 76179']


# Using .apply to find data about SEVERAL tow truck companies

The file `trucks-subset.csv` has information about the trucks, we'll use it to find the pages to scrape.

### Open up `trucks-subset.csv` and save it into a dataframe

In [93]:
df = pd.read_csv("trucks-subset.csv")
df

,TDLR Number
0,006507931C
1,006179570C
2,006502097C


## Go through each row of the dataset, displaying the URL you will need to scrape for the information on that row

You don't have to actually use the search form for each of these - look at the URL you're on, it has the number in it!

For example, one URL might look like `https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C`.

- *TIP: Use .apply and a function*
- *TIP: Unlike the Yelp example, you'll need to build this URL from pieces*
- *TIP: You probably don't want to `print` unless you're going to fix it for the next question 
- *TIP: pandas won't showing you the entire url! Run `pd.set_option('display.max_colwidth', -1)` to display aaaalll of the text in a cell*

In [94]:
df['TDLR Number']

0    006507931C
1    006179570C
2    006502097C
Name: TDLR Number, dtype: object

In [95]:
pd.set_option('display.max_colwidth', -1)
def get_url(row):
    return "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber={}".format(row['TDLR Number'])
df['url'] = df.apply(get_url, axis=1)
df.head()

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save this URL into a new column of your dataframe, called `url`

- *TIP: Use a function and `.apply`*
- *TIP: Be sure to use `return`*

In [ ]:
# unclear on how this question differs from the previous question

## Go through each row of the dataset, printing out information about each tow truck company.

Now will be **scraping** inside of your function.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

Just print it out for now.

- *TIP: use .apply*
- *TIP: You'll be using the code you wrote before, but converted into a function*
- *TIP: Remember how the TDLR Number is in the URL? You don't need to do the form submission if you don't want!*
- *TIP: Make sure you adjust any variables so you don't scrape the same page again and again*

In [96]:
def get_towing_info(row):
    
    # visit the url
    url = row['url']
    driver.get(url)

    try:
        business_name = driver.find_elements_by_tag_name('td')[5].text
        business = business_name.split(":")[1]
        print(business)
    except:
        print("Address is wonky.")

    # find owner/ operator
    try:
        owner = driver.find_elements_by_tag_name('td')[7].text
        print(owner)
    except:
        print("Owner is wonky.")

    # find phone number
    try:
        phone = driver.find_elements_by_tag_name('td')[9].text
        print(phone)
    except:
        print("Phone number is wonky.")

    # find license status
    try:
        status = driver.find_elements_by_tag_name('td')[12].text
        print(status)
    except:
        print("License status is wonky.")

    # find physical address
    try:
        address_info = driver.find_elements_by_tag_name('td')[14].text
        address = address_info.split("\n")[5:7]
        print(address)
    except:
        print("Address is wonky.")

df.apply(get_towing_info, axis=1)

   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
['1617 COUNTY ROAD 4130', 'BONHAM, TX. 75418']
   B.D. SMITH TOWING
Owner/Officer:   BRANDT SMITH / OWNER
Phone:   8173330706
Status:  Expired
['13619 BRETT JACKSON RD', 'FORT WORTH, TX. 76179']
   BARRY MICHAEL SMITH
Owner/Officer:   BARRY MICHAEL SMITH / OWNER
Phone:   8066544404
Status:  Active
['4501 W CEMETERY RD', 'CANYON, TX. 79015']


0    None
1    None
2    None
dtype: object

## Scrape the following information for each row of the dataset, and save it into new columns in your dataframe.

- The business name
- Owner/operator
- Phone number
- License status (Active, Expired, Etc)
- Physical address

It's basically what we did before, but using the function a little differently.

- *TIP: Same as above, but you'll be returning a `pd.Series` and the `.apply` line is going to be a lot longer*
- *TIP: Save it to a new dataframe!*
- *TIP: Make sure you change your `df` variable names correctly if you're cutting and pasting - there are a few so it can get tricky*

In [107]:
 def get_towing_info(row):
    
    # visit the url
    url = row['url']
    driver.get(url)

    try:
        business_name = driver.find_elements_by_tag_name('td')[5].text
        business = business_name.split(":")[1]
        print(business)
    except:
        print("Address is wonky.")

    # find owner/ operator
    try:
        owner = driver.find_elements_by_tag_name('td')[7].text
        print(owner)
    except:
        print("Owner is wonky.")

    # find phone number
    try:
        phone = driver.find_elements_by_tag_name('td')[9].text
        print(phone)
    except:
        print("Phone number is wonky.")

    # find license status
    try:
        status = driver.find_elements_by_tag_name('td')[12].text
        print(status)
    except:
        print("License status is wonky.")

    # find physical address
    try:
        address_info = driver.find_elements_by_tag_name('td')[14].text
        address = ''.join(address_info.split("\n")[5:7])
        print(address)
    except:
        print("Address is wonky.")  
    
    return pd.Series({
        'business name': business,
        'owner': owner,
        'phone_number': phone,
        'license_status': status,
        'address': address
    })

In [108]:
new_df = df.apply(get_towing_info, axis=1).join(df)
new_df

   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
1617 COUNTY ROAD 4130BONHAM, TX. 75418
   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
1617 COUNTY ROAD 4130BONHAM, TX. 75418
   B.D. SMITH TOWING
Owner/Officer:   BRANDT SMITH / OWNER
Phone:   8173330706
Status:  Expired
13619 BRETT JACKSON RDFORT WORTH, TX. 76179
   BARRY MICHAEL SMITH
Owner/Officer:   BARRY MICHAEL SMITH / OWNER
Phone:   8066544404
Status:  Active
4501 W CEMETERY RDCANYON, TX. 79015


,business name,owner,phone_number,license_status,address,TDLR Number,url
0,AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"1617 COUNTY ROAD 4130BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"13619 BRETT JACKSON RDFORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"4501 W CEMETERY RDCANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C


### Save your dataframe as a CSV named `tow-trucks-extended.csv`

In [109]:
new_df.to_csv('tow-trucks-extended.csv', index=False)

### Re-open your dataframe to confirm you didn't save any extra weird columns

In [ ]:
# done.

## Process the entire `tow-trucks.csv` file

We just did it on a short subset so far. Now try it on all of the tow trucks. **Save as the same filename as before**

In [114]:
df_add = pd.read_csv("tow-trucks.csv")
df_add

,TDLR Number
0,006507931C
1,006179570C
2,006502097C
3,006494912C
4,0649468VSF
5,006448786C
6,0648444VSF
7,0651667VSF
8,006017767C
9,006495492C


In [115]:
pd.set_option('display.max_colwidth', -1)
def get_url(row):
    return "https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber={}".format(row['TDLR Number'])
df_add['url'] = df_add.apply(get_url, axis=1)
df_add

,TDLR Number,url
0,006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,006494912C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006494912C
4,0649468VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0649468VSF
5,006448786C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006448786C
6,0648444VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0648444VSF
7,0651667VSF,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=0651667VSF
8,006017767C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006017767C
9,006495492C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006495492C


In [116]:
df_add = df_add.apply(get_towing_info, axis=1).join(df)
df_add

   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
1617 COUNTY ROAD 4130BONHAM, TX. 75418
   AUGUSTUS E SMITH
Owner/Officer:   AUGUSTUS EUGENE SMITH / OWNER
Phone:   9032276464
Status:  Active
1617 COUNTY ROAD 4130BONHAM, TX. 75418
   B.D. SMITH TOWING
Owner/Officer:   BRANDT SMITH / OWNER
Phone:   8173330706
Status:  Expired
13619 BRETT JACKSON RDFORT WORTH, TX. 76179
   BARRY MICHAEL SMITH
Owner/Officer:   BARRY MICHAEL SMITH / OWNER
Phone:   8066544404
Status:  Active
4501 W CEMETERY RDCANYON, TX. 79015
   HEATH SMITH
Owner/Officer:   HEATH A SMITH / OWNER
Phone:   940-552-0687
Status:  Expired
1529 WILBARGER STVERNON, TX. 76384
   HEATH SMITH
Owner/Officer:   HEATH A SMITH / OWNER
Phone:   9405520687
Status:  Expired
1529 WILBARGER STVERNON, TX. 76384
   HYSMITH AUTOMOTIVE
Owner/Officer:   WILLIAM THOMAS HYSMITH / PRESIDENT
Owner/Officer:   ASHLEY ERIN HYSMITH / TREASURER


   HYSMITH AUTOMOTIVE & TRUCK REPAIR INC
Owner/Officer:  

,business name,owner,phone_number,license_status,address,TDLR Number,url
0,AUGUSTUS E SMITH,Owner/Officer: AUGUSTUS EUGENE SMITH / OWNER,Phone: 9032276464,Status: Active,"1617 COUNTY ROAD 4130BONHAM, TX. 75418",006507931C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006507931C
1,B.D. SMITH TOWING,Owner/Officer: BRANDT SMITH / OWNER,Phone: 8173330706,Status: Expired,"13619 BRETT JACKSON RDFORT WORTH, TX. 76179",006179570C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006179570C
2,BARRY MICHAEL SMITH,Owner/Officer: BARRY MICHAEL SMITH / OWNER,Phone: 8066544404,Status: Active,"4501 W CEMETERY RDCANYON, TX. 79015",006502097C,https://www.tdlr.texas.gov/tools_search/mccs_display.asp?mcrnumber=006502097C
3,HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 940-552-0687,Status: Expired,"1529 WILBARGER STVERNON, TX. 76384",NaN,NaN
4,HEATH SMITH,Owner/Officer: HEATH A SMITH / OWNER,Phone: 9405520687,Status: Expired,"1529 WILBARGER STVERNON, TX. 76384",NaN,NaN
5,HYSMITH AUTOMOTIVE,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,,,NaN,NaN
6,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,Owner/Officer: HYSMITH ERIN ASHLEY / TREASURER,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,,,NaN,NaN
7,HYSMITH AUTOMOTIVE & TRUCK REPAIR INC,Owner/Officer: WILLIAM THOMAS HYSMITH / PRESIDENT,Owner/Officer: ASHLEY ERIN HYSMITH / TREASURER,,,NaN,NaN
8,JEFF & WENDY SMITH,Owner/Officer: JEFF SMITH / PARTNER,Owner/Officer: WENDY SMITH / PARTNER,,,NaN,NaN
9,JEFF SMITH,Owner/Officer: JEFFREY JOHN SMITH / OWNER,Phone: 8324354670,Status: Active,"4338 HARVEY RDCROSBY, TX. 77532",NaN,NaN


In [117]:
df_add.to_csv('tow-trucks-extended.csv', index=False)

In [ ]:
# this is soul-crushing.